## Leadboard final 
- [12.5] 
- Hybrid_Retrieve(“query”: “boost”:0.5, “knn”:”boost”:1) size =100 
- Reranking top k = 5


In [1]:
#!./elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori

In [2]:
#Suk3UCF_msp1jgipFph-


In [1]:
# 엘라스틱서치의 데몬 인스턴스 만들기
# Nori 설치 이전에 데몬을 생성하면 Nori가 바로 사용할 수 없음, 이때는 데몬 재실행 필요

import os
from elasticsearch import Elasticsearch, helpers
import numpy as np
import pandas as pd
import json
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(['/data/ephemeral/home/elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

# 인스턴스를 로드하는 데 약간의 시간이 걸림
import time
time.sleep(30)


In [2]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

daemon   1184357 1184339  7 06:17 ?        00:00:02 /data/ephemeral/home/elasticsearch-8.8.0/jdk/bin/java -Xms4m -Xmx64m -XX:+UseSerialGC -Dcli.name=server -Dcli.script=/data/ephemeral/home/elasticsearch-8.8.0/bin/elasticsearch -Dcli.libs=lib/tools/server-cli -Des.path.home=/data/ephemeral/home/elasticsearch-8.8.0 -Des.path.conf=/data/ephemeral/home/elasticsearch-8.8.0/config -Des.distribution.type=tar -cp /data/ephemeral/home/elasticsearch-8.8.0/lib/*:/data/ephemeral/home/elasticsearch-8.8.0/lib/cli-launcher/* org.elasticsearch.launcher.CliToolLauncher
daemon   1184415 1184357 99 06:17 ?        00:00:50 /data/ephemeral/home/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -D

In [3]:
username = 'elastic'

# 위 명령 실행 결과의 마지막 부분인 PASSWORD elastic 값으로 교체 필요
password = 'password'

es = Elasticsearch(['https://localhost:9200'], basic_auth=(username, password), ca_certs="/data/ephemeral/home/elasticsearch-8.8.0/config/certs/http_ca.crt")

resp = dict(es.info())

resp

{'name': 'instance-11887',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'Qi0bps3KR76S74__mRoyvA',
 'version': {'number': '8.8.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f',
  'build_date': '2023-05-23T17:16:07.179039820Z',
  'build_snapshot': False,
  'lucene_version': '9.6.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [4]:
import os
import json
from elasticsearch import Elasticsearch, helpers
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Sentence Transformer 모델 초기화 (한국어 임베딩 생성 가능한 어떤 모델도 가능)
# model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# SetntenceTransformer를 이용하여 임베딩 생성
def get_embedding(sentences):
    return model.encode(sentences)


# 주어진 문서의 리스트에서 배치 단위로 임베딩 생성
def get_embeddings_in_batches(docs, batch_size=100):
    batch_embeddings = []
    for i in range(0, len(docs), batch_size):
        batch = docs[i:i + batch_size]
        contents = [doc["content"] for doc in batch]
        embeddings = get_embedding(contents)
        batch_embeddings.extend(embeddings)
        print(f'batch {i}')
    return batch_embeddings


# 새로운 index 생성
def create_es_index(index, settings, mappings):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, settings=settings, mappings=mappings)


# 지정된 인덱스 삭제
def delete_es_index(index):
    es.indices.delete(index=index)


# Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    return helpers.bulk(es, actions)


# 역색인을 이용한 검색
def sparse_retrieve(query_str, size):
    query = {
        "match": {
            "content": {
                "query": query_str
            }
        }
    }
    return es.search(index="test", query=query, size=size, sort="_score")


# Vector 유사도를 이용한 검색
def dense_retrieve(query_str, size):
    # 벡터 유사도 검색에 사용할 쿼리 임베딩 가져오기
    query_embedding = get_embedding([query_str])[0]

    # KNN을 사용한 벡터 유사성 검색을 위한 매개변수 설정
    knn = {
        "field": "embeddings",
        "query_vector": query_embedding.tolist(),
        "k": size,
        "num_candidates": 100
    }

    # 지정된 인덱스에서 벡터 유사도 검색 수행
    return es.search(index="test", knn=knn)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [70]:
def hybrid_retrieve(query_str, size):
    query_embedding = get_embedding([query_str])[0]
    
    body = {
        "query" : {
            "match" : {
                "content": {
                    "query": query_str,
                    "boost" : 0.5 
                }
            }
        },
    "knn": {
        "field": "embeddings",
        "query_vector" : query_embedding.tolist(),
        "k" : 5,
        "num_candidates" : 50,
        "boost": 1
    },
    "size": size 
    }
    return es.search(index="test", body = body)

In [24]:

# 색인을 위한 setting 설정
settings = {
    "analysis": {
        "analyzer": {
            "nori": {
                "type": "custom",
                "tokenizer": "nori_tokenizer",
                "decompound_mode": "mixed",
                "filter": ["nori_posfilter"]
            }
        },
        "filter": {
            "nori_posfilter": {
                "type": "nori_part_of_speech",
                # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
            }
        }
    }
}

# 색인을 위한 mapping 설정 (역색인 필드, 임베딩 필드 모두 설정)
mappings = {
    "properties": {
        # 역색인 필드 
        "content": {"type": "text", "analyzer": "nori"}, 
        # 임베딩 필드 
        "embeddings": {
            "type": "dense_vector",
            "dims": 768,
            "index": True,
            "similarity": "l2_norm"
        }
    }
}

In [25]:
create_es_index("test", settings, mappings)

In [26]:
# 문서의 content 필드에 대한 임베딩 생성
index_docs = []
with open("../data/documents.jsonl") as f:
    docs = [json.loads(line) for line in f]
embeddings = get_embeddings_in_batches(docs)

""" 
docs[0] -> 
{'docid': '42508ee0-c543-4338-878e-d98c6babee66',
 'src': 'ko_mmlu__nutrition__test',
 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. ....
 }
"""

batch 0
batch 100
batch 200
batch 300
batch 400
batch 500
batch 600
batch 700
batch 800
batch 900
batch 1000
batch 1100
batch 1200
batch 1300
batch 1400
batch 1500
batch 1600
batch 1700
batch 1800
batch 1900
batch 2000
batch 2100
batch 2200
batch 2300
batch 2400
batch 2500
batch 2600
batch 2700
batch 2800
batch 2900
batch 3000
batch 3100
batch 3200
batch 3300
batch 3400
batch 3500
batch 3600
batch 3700
batch 3800
batch 3900
batch 4000
batch 4100
batch 4200


" \ndocs[0] -> \n{'docid': '42508ee0-c543-4338-878e-d98c6babee66',\n 'src': 'ko_mmlu__nutrition__test',\n 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. ....\n }\n"

In [27]:
# doc = docs[0], len(embedding) = 768
# doc_key = docid, src, content
for doc, embedding in zip(docs, embeddings):  
    doc['embeddings'] = embedding.tolist() 
    index_docs.append(doc)
    
# index_docs_key = docid, src, content, embeddings 

In [28]:
ret = bulk_add("test", index_docs)

In [29]:
ret

(4272, [])

In [60]:
test_query = "차량의 연비가 좋아질때 나타나는 긍정적인 효과는?"
# 역색인을 사용하는 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'source:', rst['_source']["content"])


score: 7.190627 source: 전기-가솔린 하이브리드 차량은 주로 더 나은 연비를 제공하기 위해 설계되었습니다. 이러한 차량은 가솔린 구동 차량과 비교하여 유사한 크기를 가지고 있으며, 전기 모터와 가솔린 엔진을 결합하여 동작합니다. 이러한 조합은 전기 모터의 효율성과 가솔린 엔진의 힘을 최대한 활용하여 연비를 향상시킵니다. 전기-가솔린 하이브리드 차량은 주로 도시 주행이나 교통 체증이 많은 지역에서 효과적으로 사용됩니다. 또한, 이러한 차량은 환경 친화적이며, 대기 오염을 줄이고 탄소 배출을 감소시킵니다. 따라서, 전기-가솔린 하이브리드 차량은 더 나은 연비를 제공하여 경제적이고 환경에 친화적인 선택입니다.
score: 5.721682 source: 하이브리드 자동차 운전은 대기 오염을 줄이는 데 도움이 됩니다. 하이브리드 자동차는 전기 모터와 연료 엔진을 결합하여 작동하는데, 이는 연료 소비를 줄이고 대기 중에 배출되는 유해 물질의 양을 감소시킵니다. 이러한 차량은 일반적인 연료 자동차보다 더 많은 연비를 제공하며, 이는 대기 오염을 줄이는 데 큰 영향을 미칩니다. 또한, 하이브리드 자동차는 주행 중에 발생하는 소음을 줄여 주므로 도시 환경에서의 소음 오염도 감소시킵니다. 따라서 하이브리드 자동차 운전은 환경 보호를 위한 효과적인 선택입니다.
score: 5.262266 source: 새로운 기술들은 자동차 제조업체들이 저연비의 하이브리드 차량을 출시할 수 있게 했습니다. 하지만, 내용 연수가 지난 후 하이브리드 차량 폐기는 문제를 초래합니다. 하이브리드 차량에 국한된 폐기 문제에 대해 고민해볼만한 질문은 '배터리를 재활용하기 위해 무엇을 해야 할까요?'입니다. 하이브리드 차량의 배터리는 폐기되었을 때 환경 오염을 초래할 수 있으므로, 적절한 재활용 방법을 찾는 것이 중요합니다. 배터리 재활용을 위해 어떤 기술이나 시스템을 도입해야 할지 고민해보는 것이 필요합니다. 또한, 배터리 재활용을 위한 정책과 규제도 필요할 수 있습니다. 이러한 문제에 

/tmp/ipykernel_1184339/2553866457.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body = body)


In [54]:
test_query = "차량의 연비가 좋아질때 나타나는 긍정적인 효과는?"
# 역색인을 사용하는 검색 예제
search_result_retrieve = hybrid_retrieve(test_query, 3)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print('score:', rst['_score'], 'docid: ',rst["_source"]["docid"], 'source:', rst['_source']["content"])

score: 0.0482685 docid:  abf99ff1-d6bf-4020-b752-da7cb8611915 source: 전기-가솔린 하이브리드 차량은 주로 더 나은 연비를 제공하기 위해 설계되었습니다. 이러한 차량은 가솔린 구동 차량과 비교하여 유사한 크기를 가지고 있으며, 전기 모터와 가솔린 엔진을 결합하여 동작합니다. 이러한 조합은 전기 모터의 효율성과 가솔린 엔진의 힘을 최대한 활용하여 연비를 향상시킵니다. 전기-가솔린 하이브리드 차량은 주로 도시 주행이나 교통 체증이 많은 지역에서 효과적으로 사용됩니다. 또한, 이러한 차량은 환경 친화적이며, 대기 오염을 줄이고 탄소 배출을 감소시킵니다. 따라서, 전기-가솔린 하이브리드 차량은 더 나은 연비를 제공하여 경제적이고 환경에 친화적인 선택입니다.
score: 0.036059547 docid:  59d5d7bb-6700-40ad-8884-ff43b1a9a1a0 source: 오늘날 많은 자동차들은 과거에 제작된 것보다 더 나은 연비를 얻도록 설계되었습니다. 이러한 변화는 자원 보존의 필요성으로 인해 발생하였습니다. 자원 보존은 우리가 가진 자원을 효율적으로 사용하고 보호하는 것을 의미합니다. 과거에는 연료가 풍부하게 존재하였기 때문에 자동차의 연비에 큰 관심이 없었습니다. 그러나 현대 사회에서는 에너지 자원의 고갈과 환경 문제로 인해 연비 개선이 필수적이 되었습니다. 자동차 제조사들은 이러한 필요성을 인식하고, 엔진과 디자인을 개선하여 더 효율적인 연비를 얻을 수 있도록 노력하였습니다. 이제 우리는 더 적은 연료로 더 멀리 갈 수 있는 자동차를 이용할 수 있으며, 이는 자원 보존에 큰 도움이 됩니다. 또한, 연비 개선은 환경에도 긍정적인 영향을 미칩니다. 더 효율적인 연비는 대기 오염과 온실가스 배출량을 줄여주어 더 깨끗한 환경을 만들어줍니다. 따라서, 자동차 제조사들은 자원 보존과 환경 보호를 위해 더 나은 연비를 얻을 수 있도록 자동차를 설계하고 있습니다.
score: 0.028608412 docid:

/tmp/ipykernel_1184339/2797902526.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body = body)


## RAG 구현 

In [17]:
# - 사용자가 대화를 통해 과학 지식에 관한 주제로 질문하면 search api를 호출할 수 있어야 한다.
# - 과학 상식과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.

In [74]:
# 아래부터는 실제 RAG를 구현하는 코드입니다.
from openai import OpenAI
import traceback

# OpenAI API 키를 환경변수에 설정
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

client = OpenAI()
# 사용할 모델을 설정(여기서는 gpt-3.5-turbo-1106 모델 사용)
llm_model = "gpt-3.5-turbo-1106"

# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 과학 상식 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 한국어로 답변을 생성한다.
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 과학 상식 전문가

## Instruction
- 사용자가 대화를 통해 지식에 관한 주제로 질문하면 반드시 search api를 호출할 수 있어야 한다.
- 지식과 관련되지 않은 나머지 대화 메시지에는 함수 호출 없이 적절한 대답을 생성한다.
"""



In [19]:
# Function calling에 사용할 함수 정의
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "search",
#             "description": "search relevant documents",
#             "parameters": {
#                 "properties": {
#                     "standalone_query": {
#                         "type": "string",
#                         "description": "Final query suitable for use in search from the user messages history."
#                     }
#                 },
#                 "required": ["standalone_query"],
#                 "type": "object"
#             }
#         }
#     },
# ]

In [75]:
# Function calling에 사용할 함수 정의
tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search relevant documents",
            "parameters": {
                "properties": {
                    "long_question": {
                        "type": "string",
                        # "description": "Final query suitable for use in search from the user messages history"
                        # "description": "Keywords suitable for use in search engine."
                        # "description": "User's question in Korean, including all the keywords in the user messages"
                        "description": "User's question in Korean. Full message if the user message is single-turn."}
                        #"description" : "User's question in Korean. Full message if the user message is single-turn. Additionally, categorize the question and infer user's intent."}
                },
                "required": ["long_question"],
                "type": "object"
            }
        }
    },
]

In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
import json

# Ranker 모델 로드
model_path = 'Dongjin-kr/ko-reranker'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model_reranker = AutoModelForSequenceClassification.from_pretrained(model_path)
model_reranker.eval()

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

In [68]:
# 상위 3개 

def rerank_with_model(query, retrieved_docs):
    # Query와 검색된 문서를 쌍으로 만듬
    pairs = [[query, doc["_source"]["content"]] for doc in retrieved_docs]
    
    # Ranker 모델을 통해 점수 계산
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model_reranker(**inputs, return_dict=True).logits.view(-1, ).float()
        normalized_scores = exp_normalize(scores.numpy())

    # 점수에 따라 문서를 재정렬하고 상위 3개의 문서만 남김
    reranked_docs = sorted(zip(retrieved_docs, normalized_scores), key=lambda x: x[1], reverse=True)[:5]
    return reranked_docs

In [76]:
# LLM과 검색엔진을 활용한 RAG 구현
def answer_question(messages):
    # 함수 출력 초기화
    response = {"standalone_query": "", "topk": [], "references": [], "answer": ""}

    # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
    msg = [{"role": "system", "content": persona_function_calling}] + messages
    try:
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            #tool_choice={"type": "function", "function": {"name": "search"}},
            temperature=0,
            seed=1,
            timeout=10
        )
    except Exception as e:
        traceback.print_exc()
        return response

    # 검색이 필요한 경우 검색 호출 후 결과를 활용하여 답변 생성
    if result.choices[0].message.tool_calls:
        tool_call = result.choices[0].message.tool_calls[0]
        function_args = json.loads(tool_call.function.arguments)
        standalone_query = function_args.get("long_question")

        # 검색 결과 추출
        search_result = hybrid_retrieve(standalone_query, 100)

        # 검색 결과를 Ranker 모델로 재정렬
        reranked_docs = rerank_with_model(standalone_query, search_result['hits']['hits'])

        response["standalone_query"] = standalone_query
        retrieved_context = []
        for doc, score in reranked_docs:
            retrieved_context.append(doc["_source"]["content"])
            response["topk"].append(doc["_source"]["docid"])
            response["references"].append({"score": float(score), "content": doc["_source"]["content"]})

        # 재정렬된 문서들을 LLM에 다시 넣어 답변 생성
        # messages.append({"role": "assistant", "content": json.dumps(retrieved_context)})
        # msg = [{"role": "system", "content": persona_qa}] + messages
        # try:
        #     qaresult = client.chat.completions.create(
        #             model=llm_model,
        #             messages=msg,
        #             temperature=0,
        #             seed=1,
        #             timeout=30
        #         )
        # except Exception as e:
        #     traceback.print_exc()
        #     return response
        # response["answer"] = qaresult.choices[0].message.content

    # 검색이 필요하지 않은 경우 바로 답변 생성
    else:
        response["answer"] = result.choices[0].message.content

    return response


In [77]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as f, open(output_filename, "w") as of:
        idx = 0
        for line in f:
            j = json.loads(line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": j["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag("../data/eval.jsonl", "../submission/[12.5]submission.csv")


Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


/tmp/ipykernel_1184339/2553866457.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  return es.search(index="test", body = body)


Answer: 

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Answer: 

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
Answer: 

Test 4
Question: [{'role': 'user', 'content': 'Dmitri Ivanovsky가 누구야?'}]
Answer: 

Test 5
Question: [{'role': 'user', 'content': '피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?'}]
Answer: 

Test 6
Question: [{'role': 'user', 'content': '헬륨이 다른 원소들과 반응을 잘 안하는 이유는?'}]
Answer: 

Test 7
Question: [{'role': 'user', 'content': '문맹 비율이 사회 발전에 미치는 영향은?'}]
Answer: 

Test 8
Question: [{'role': 'user', 'content': '이란 콘트라 사건이 뭐야'}, {'role': 'assistant', 'content': '이란-콘트라 사건은 로널드 레이건 집권기인 1986년에 레이건 행정부와 CIA가 적성국이었던 이란에게 무기를 몰래 수출한 대금으로 니카라과의 우익 성향 반군 콘트라를 지원하면서 동시에 반군으로부터 마약을 사들인 후 미국에 판매하다가 발각되어 큰 파장을 일으킨 사건입니다.'}, {'role': 'user', 'content': '이 사건이 미국 정치에 미친 영향